# Object Detection on Vehicle Dataset Images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Pytorch implementation of YoloV5 object detection
* Installations and Imports
* Dataset Loading
* Model Loading
* Training model
* Evaluation
* Inference


### Importing Necessary Modules

In [ ]:
%cd "/content/drive/MyDrive/SEM 1/TML/yolov5"

/content/drive/.shortcut-targets-by-id/1LqhIB71FrBRNw-WAPjOHPWbbEnFt7cvQ/SEM 1/TML/yolov5


In [ ]:
!pip install utils

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone

In [ ]:
# %cd yolov5
%pip install -qr requirements.txt  # install
import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 2023-11-25 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)


In [ ]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import json
import urllib
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

np.random.seed(42)

## Dataset Collection
We have used Vehicle Dataset by  Yudha Bhakti Nugraha and Kris - https://universe.roboflow.com/roboflow-100/vehicles-q0x2v

It contains the data of around 4-5k images, divided into train, val and test set. With a json file for each containing image information like bounding boxes and classes.

### Data convert to yolo format annotation

### Each image have a txt file containing the information in following order - class_id, x_centre, y_centre, width, height

In [ ]:
import json
import os
import argparse

count_dict = {}
class COCO2YOLO:
    def __init__(self, json_file, output):
        self._check_file_and_dir(json_file, output)
        self.labels = json.load(open(json_file, 'r', encoding='utf-8'))
        self.coco_id_name_map = self._categories()
        self.coco_name_list = list(self.coco_id_name_map.values())
        self.output = output
        print("total images", len(self.labels['images']))
        print("total categories", len(self.labels['categories']))
        print("total labels", len(self.labels['annotations']))

    def _check_file_and_dir(self, file_path, dir_path):
        if not os.path.exists(file_path):
            raise ValueError("file not found")
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    def _categories(self):
        categories = {}
        for cls in self.labels['categories']:
            categories[cls['id']] = cls['name']
        return categories

    def _load_images_info(self):
        images_info = {}
        for image in self.labels['images']:
            id = image['id']
            file_name = image['file_name']
            if file_name.find('\\') > -1:
                file_name = file_name[file_name.index('\\')+1:]
            w = image['width']
            h = image['height']
            images_info[id] = (file_name, w, h)

        return images_info

    def _bbox_2_yolo(self, bbox, img_w, img_h):
        x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
        centerx = bbox[0] + w / 2
        centery = bbox[1] + h / 2
        dw = 1 / img_w
        dh = 1 / img_h
        centerx *= dw
        w *= dw
        centery *= dh
        h *= dh
        return centerx, centery, w, h

    def _convert_anno(self, images_info):
        anno_dict = dict()
        for anno in self.labels['annotations']:
            bbox = anno['bbox']
            image_id = anno['image_id']
            category_id = anno['category_id']

            image_info = images_info.get(image_id)
            image_name = image_info[0]
            img_w = image_info[1]
            img_h = image_info[2]
            yolo_box = self._bbox_2_yolo(bbox, img_w, img_h)

            anno_info = (image_name, category_id, yolo_box)
            anno_infos = anno_dict.get(image_id)
            if not anno_infos:
                anno_dict[image_id] = [anno_info]
            else:
                anno_infos.append(anno_info)
                anno_dict[image_id] = anno_infos
        return anno_dict

    def save_classes(self):
        sorted_classes = list(map(lambda x: x['name'], sorted(self.labels['categories'], key=lambda x: x['id'])))
        print('coco names', sorted_classes)
        with open('coco.names', 'w', encoding='utf-8') as f:
            for cls in sorted_classes:
                f.write(cls + '\n')
        f.close()

    def coco2yolo(self):
        print("loading image info...")
        images_info = self._load_images_info()
        print("loading done, total images", len(images_info))

        print("start converting...")
        anno_dict = self._convert_anno(images_info)
        print("converting done, total labels", len(anno_dict))

        print("saving txt file...")
        self._save_txt(anno_dict)
        print("saving done")

    def _save_txt(self, anno_dict):
        for k, v in anno_dict.items():
            file_name = os.path.splitext(v[0][0])[0] + ".txt"
            with open(os.path.join(self.output, file_name), 'w', encoding='utf-8') as f:
                print(k, v)
                for obj in v:
                    cat_name = self.coco_id_name_map.get(obj[1])
                    category_id = self.coco_name_list.index(cat_name)
                    box = ['{:.6f}'.format(x) for x in obj[2]]
                    box = ' '.join(box)
                    line = str(category_id) + ' ' + box
                    f.write(line + '\n')
                    if category_id not in count_dict:
                        count_dict[category_id] = 1
                    else:
                        count_dict[category_id] += 1
        print(count_dict)

In [ ]:
json_root_path = '/content/drive/MyDrive/SEM 1/TML/data/vehicles.v2-release.coco/annotations'
save_root_path = '/content/drive/MyDrive/SEM 1/TML/data/yolo_data/labels'
## For training data
dtype = 'train'
json_file = os.path.join(json_root_path, dtype + '_annotations_coco.json')
save_path = os.path.join(save_root_path, dtype)
c2y = COCO2YOLO(json_file, save_path)
c2y.coco2yolo()

## For validation data
dtype = 'val'
json_file = os.path.join(json_root_path, dtype + '_annotations_coco.json')
save_path = os.path.join(save_root_path, dtype)
c2y = COCO2YOLO(json_file, save_path)
c2y.coco2yolo()

## For testing data
dtype = 'test'
json_file = os.path.join(json_root_path, dtype + '_annotations_coco.json')
save_path = os.path.join(save_root_path, dtype)
c2y = COCO2YOLO(json_file, save_path)
c2y.coco2yolo()

### Making config files

#### Storing the below config in data.yaml file, which will be passed to training module

In [ ]:
train: /content/drive/MyDrive/SEM 1/TML/data/yolo_data/images/train/
val:  /content/drive/MyDrive/SEM 1/TML/data/yolo_data/images/val/
test: /content/drive/MyDrive/SEM 1/TML/data/yolo_data/images/test

# number of classes
nc: 13

# class names
names:
  0: vehicles
  1: big bus
  2: big truck
  3: bus-l-
  4: bus-s-
  5: car
  6: mid truck
  7: small bus
  8: small truck
  9: truck-l-
  10: truck-m-
  11: truck-s-
  12: truck-xl-

### Training yolov5 small on default parameters
### Arguments to be passed -
* img - Size in which images are to be resized according to resolution requirement
* batch - Batch size
* epochs - Number of epochs
* data - Path to data config file
* weights - Path to pre-trained model

In [ ]:
!python train.py --img 640 --batch 32 --epochs 300 --data data/data.yaml --weights "/content/drive/MyDrive/SEM 1/TML/yolov5/weights/yolov5s.pt"

### Training yolov5 small on with freezing the backbone architecture

In [ ]:
!python train.py --img 640 --batch 32 --epochs 100 --data data/data.yaml --weights "/content/drive/MyDrive/SEM 1/TML/yolov5/weights/yolov5s.pt" --cfg models/yolov5n.yaml --freeze 10

### Training yolov5 small on with freezing all the layers

In [ ]:
!python train.py --img 640 --batch 32 --epochs 100 --data data/data.yaml --weights "/content/drive/MyDrive/SEM 1/TML/yolov5/weights/yolov5s.pt" --cfg models/yolov5n.yaml --freeze 24

## Evaluation on Testing Data

### Arguments to be passed -
* img - Size in which images are to be resized according to resolution requirement
* data - Path to data config file
* weights - Path to the best saved model from training

In [ ]:
!python val.py --weights "/content/drive/MyDrive/SEM 1/TML/yolov5/runs/train/exp7_90epochs/weights/best.pt" --data data/data.yaml --img 640

### Inference on Single image

### Arguments to be passed -
* img - Size in which images are to be resized according to resolution requirement
* data - Path to data config file
* weights - Path to the best saved model from training
* conf - to set confidence threshold
* source - Path to image
* line-thichness = to make small boundaries while plotting the predictions

In [ ]:
!python detect.py --weights "/content/drive/MyDrive/SEM 1/TML/yolov5/runs/train/exp3_30epochs_yolos/weights/best.pt" --img 640 --conf 0.25 --source "/content/drive/MyDrive/SEM 1/TML/data/yolo_data/images/test/adit_mp4-1002_jpg.rf.5e4018e963af1251b3f7e6fd487c479e.jpg" --line-thickness 1